In [1]:
# Importing public libraries
import os
import time
import argparse
import epynet
import yaml
import torch
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.utils import from_networkx
from sklearn.model_selection import train_test_split

In [2]:
# Importing custom libraries
from utils.epanet_loader import get_nx_graph
from utils.visualisation import visualise
from utils.epanet_simulator import epanetSimulator
from utils.data_loader import battledimLoader, dataCleaner, dataGenerator, embedSignalOnGraph, rescaleSignal, predictionTaskDataSplitter
from utils.early_stopping import EarlyStopping
from modules.torch_gnn import ChebNet
import utils.user_interface as ui

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3070 Ti Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [7]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("\nRunning: \t 'import_model.py' ")
print("Using device: \t {}".format(device))

# Hardcoded configurations
wdn = 'l-town'  # Choice of water-distribution network
gnn = 'chebnet'  # Choice of GNN model
# weights = 'unweighted'  # Edge weights type
weights = 'pipe_length'  # Edge weights type
visualise = 'yes'  # Visualise the graph
visualiseWhat = 'sensor_location'  # What to visualise
scaling = 'minmax'  # Rescaling method
tag = 'predictor'  # Descriptive tag
epochs = 100  # Number of training epochs
timesteps = 3  # Number of timesteps in the signal

self_loop = True 

print("\nConfigurations for session: \n" + 22*"-")
print("WDN: {}\nGNN: {}\nWeights: {}\nVisualise: {}\nVisualise What: {}\nScaling: {}\nTag: {}\nEpochs: {}\n".format(
    wdn, gnn, weights, visualise, visualiseWhat, scaling, tag, epochs))



Running: 	 'import_model.py' 
Using device: 	 cuda

Configurations for session: 
----------------------
WDN: l-town
GNN: chebnet
Weights: pipe_length
Visualise: yes
Visualise What: sensor_location
Scaling: minmax
Tag: predictor
Epochs: 100



In [8]:
# Set the filepaths for the execution
print('Setting environment paths...\n')

path_to_data   = './data/' + wdn + '-data/'       # Datasets are stored here
path_to_wdn    = './data/' + wdn.upper() + '.inp' # EPANET input file
path_to_logs   = './studies/logs/'                     # Log directory
path_to_figs   = './studies/figs/'                     # Figure directory
path_to_models = './studies/models/'                   # Saved models directory               

execution_no   = 1                                                                                      # Initialise execution ID number
execution_id   = wdn + '-' + gnn + '-' + tag + '-' + weights + '-' + scaling + '-' + '{}'.format('self_loop-' if self_loop else '')+ '-' + str(timesteps)  # Initialise execution ID name
logs           = [log for log in os.listdir(path_to_logs) if log.endswith('.csv')]                      # Load all logs in directory to list

while execution_id + str(execution_no) + '.csv' in logs:    # For every matching file name in the directory
    execution_no += 1                                       # Increment the execution id number

execution_id   = execution_id + str(execution_no)           # Update the execution ID
model_path     = path_to_models + execution_id + '.pt'      # Generate complete model path w/ filename
log_path       = path_to_logs   + execution_id + '.csv'     # Generate complete log path w/ filename

# If we have already trained a similar model we may wish to load its weights
# So, we must also generate a path to that model's state dictionary
if execution_no > 1:
    last_id         = wdn + '-' + gnn + '-' + tag + '-' # Initialise previous version execution ID name
    last_id         = last_id + str(execution_no - 1)                  # Execution ID number is the current number - 1  
    last_model_path = path_to_models + last_id + '.pt'                 # Generate complete path to the previously trained model
    last_log_path   = path_to_logs   + last_id + '.csv'                # Generate complete path to the previous 
    

Setting environment paths...



In [9]:
print('Importing EPANET file and converting to graph...\n')

# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
G , pos , head = get_nx_graph(wdn, weight_mode=weights, get_head=True)

Importing EPANET file and converting to graph...



In [10]:
print('Importing dataset configuration...\n')

# Open the dataset configuration file
with open(path_to_data + 'dataset_configuration.yaml') as file:
    
    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader) 

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

Importing dataset configuration...



In [11]:
for sensor_node in sensors:
    G.add_edge(u_of_edge=sensor_node,
                v_of_edge=sensor_node,
                weight=1.,name='SELF')

In [12]:
print('Running EPANET simulation to generate nominal pressure data...\n')
nominal_wdn_model = epanetSimulator(path_to_wdn, path_to_data)
nominal_wdn_model.simulate()
nominal_pressure = nominal_wdn_model.get_simulated_pressure()

Running EPANET simulation to generate nominal pressure data...



In [ ]:
nominal_pressure.shape

In [13]:
print('Pre-processing nominal pressure data for training...\n')

# Populate feature vector x and label vector y from the nominal pressures
# Also retrieve the scale and bias of the scaling transformation
# This is so we can inverse transform the predicted values to calculate
# relative reconstruction errors
x,y,scale,bias = dataCleaner(pressure_df    = nominal_pressure, # Pass the nodal pressures
                             observed_nodes = sensors,          # Indicate which nodes have sensors
                             rescale        = scaling,          # Perform scaling on the timeseries data
                             task           = 'prediction',
                             mode           = 'n_timesteps',
                             n_timesteps    = 3)     

# Split the data into training and validation sets
x_trn, x_val, y_trn, y_val = train_test_split(x, y, 
                                              test_size    = 0.2,
                                              random_state = 1,
                                              shuffle      = False)

# # Now, we change the data so it matches a prediction task, where t, t-1, ..., t-n in x refer to t+1 in y
# x_p, y_p = predictionTaskDataSplitter(x, y, timesteps)                  # <------------------------------------------------- Note this is where we are using the timesteps

# # Split the data into training and validation sets
# x_trn, x_val, y_trn, y_val = train_test_split(x_p, y_p,
#                                                 test_size    = 0.2,
#                                                 random_state = 1,
#                                                 shuffle      = False)
         

Pre-processing nominal pressure data for training...



In [43]:
y.shape

(2014, 782, 1)

In [15]:
x_trn.shape

(1611, 782, 3)

In [29]:
x_trn[0,3,2]

0.1833793

In [42]:
y_trn[0,3,0]

0.1842804

In [35]:
y_trn.shape

(1611, 782, 1)

In [17]:
print('Setting up training session and creating model...\n')

# ----------------
# Hyper-parameters
# ----------------
batch_size    = 40
learning_rate = 3e-4
decay         = 6e-6
shuffle       = False
epochs        = epochs

Setting up training session and creating model...



In [18]:
# Instantiate training and test set data generators
trn_gnrtr = dataGenerator(G, x_trn, y_trn, batch_size, shuffle)
val_gnrtr = dataGenerator(G, x_val, y_val, batch_size, shuffle)

c:\Users\30806\anaconda3\envs\Msc\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [19]:
# Instantiate a Chebysev Network GNN model
model     = ChebNet(name           = 'ChebNet',
                    data_generator = trn_gnrtr,
                    device         = device,
                    in_channels    = np.shape(x_trn)[-1],
                    out_channels   = np.shape(y_trn)[-1],
                    data_scale     = scale,
                    data_bias      = bias).to(device)
        

In [20]:
# Instantiate an optimizer
optimizer = torch.optim.Adam([dict(params=model.conv1.parameters(), weight_decay=decay),
                            dict(params=model.conv2.parameters(), weight_decay=decay),
                            dict(params=model.conv3.parameters(), weight_decay=decay),
                            dict(params=model.conv4.parameters(), weight_decay=0)],
                            lr  = learning_rate,
                            eps = 1e-7)

# Configure an early stopping callback
estop    = EarlyStopping(min_delta=.00001, patience=epochs)       # By setting patience as # of epochs we make sure early stopping is never initiated


In [21]:
print("+------------------+")
print("| Training session |")
print("| Timesteps: {}    |".format(timesteps))
print("+------------------+\n")

print("Training starting...\n")

# Train for the predefined number of epochs
for epoch in range(1, epochs+1):
    
    # Start a stopwatch timer
    start_time = time.time()
    
    # Train a single epoch, passing the optimizer and current epoch number
    model.train_one_epoch(optimizer = optimizer)
    
    # Validate the model after the gradient update
    model.validate()
    
    # Update the model results for the current epoch
    model.update_results()
    
    # Print stats for the epoch and the execution time
    model.print_stats(time.time() - start_time)
    
    # If this is the best model
    if model.val_loss < model.best_val_loss:
        # We save it
        torch.save(model.state_dict(), model_path)
    
    # If model is not improving
    if estop.step(torch.tensor(model.val_loss)):
        print('Early stopping activated...')
        break

print("\nSaving training results to '{}'...\n".format(log_path))
model.results.to_csv(log_path)

+------------------+
| Training session |
| Timesteps: 3    |
+------------------+

Training starting...

epoch   trn_loss      val_loss    val_rel_err  val_rel_err_o val_rel_err_h run_time
  1     0.022993      0.002237      0.038143      0.044432      0.037866    40.04  sec
  2     0.001497      0.001073      0.020058      0.020791      0.020025    39.08  sec
  3     0.000876      0.000651      0.014846      0.012750      0.014938    39.38  sec
  4     0.000511      0.000370      0.013743      0.013412      0.013758    39.48  sec
  5     0.000253      0.000177      0.009681      0.010130      0.009661    39.63  sec
  6     0.000129      0.000086      0.006529      0.006944      0.006511    39.72  sec
  7     0.000082      0.000083      0.007313      0.007980      0.007283    39.62  sec
  8     0.000072      0.000058      0.006091      0.006429      0.006076    39.61  sec
  9     0.000076      0.000066      0.006742      0.008259      0.006675    39.65  sec
 10     0.000067      0.000